<a href="https://colab.research.google.com/github/Matanrot/BI-Project/blob/YuvalNini/SQL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

!pip install pyspark
os.environ["PYSPARK_PYTHON"] = sys.executable


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=b2ed2de84e978cc292a3eca952e0c671af48028e82cc0c413b106f389dee3702
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import sum

# SparkConf - Configuration Before SQL & Content
conf = SparkConf().setAppName("firstApp").setMaster("local[*]") 
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
df = sqlContext.read.options(header='True', inferSchema='True').csv("Glassdoor Gender.csv")
df.show(10)


+-------------------+------+---+--------+-----------+--------------+---------+-----------+----------+-------------+-----------+
|           JobTitle|Gender|Age|PerfEval|  Education|          Dept|Seniority| Salary($) | Bonus($) |Require-level|ID employee|
+-------------------+------+---+--------+-----------+--------------+---------+-----------+----------+-------------+-----------+
|   Graphic Designer|Female| 18|       5|    College|    Operations|        2|     42,363|     9,938|       Junior|       6789|
|Warehouse Associate|Female| 19|       4|        PhD|Administration|        5|     90,208|     9,268|       medium|       6790|
|                 IT|Female| 20|       5|        PhD|    Operations|        4|     70,890|    10,126|       medium|       6791|
|   Graphic Designer|Female| 20|       5|    College|         Sales|        4|     67,585|    10,541|       medium|       6792|
|   Graphic Designer|Female| 33|       5|High School|   Engineering|        5|    112,976|     9,836|   

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, when
from pyspark.sql.window import Window

# Create a SparkSession
spark = SparkSession.builder.appName("Glassdoor").getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.csv("Glassdoor Gender.csv", header=True, inferSchema=True)

# Remove leading and trailing spaces from column names
df = df.toDF(*(c.strip() for c in df.columns))

# Register the DataFrame as a temporary table
df.createOrReplaceTempView("GlassdoorGenderTable")

# Query 1 : Gender wage gap by industry

# Calculate average salary, average male salary, average female salary, and wage gap
windowSpec = Window.partitionBy("Dept")

result = df.withColumn("Avg_Salary", avg("Salary($)").over(windowSpec)) \
    .withColumn("Avg_Male_Salary", avg(when(col("Gender") == "Male", col("Salary($)"))).over(windowSpec)) \
    .withColumn("Avg_Female_Salary", avg(when(col("Gender") == "Female", col("Salary($)"))).over(windowSpec)) \
    .withColumn("Wage_Gap", col("Avg_Male_Salary") - col("Avg_Female_Salary"))

# Show the result
result.show()



In [ ]:
# Query 2 : Gender wage gap by job type

# Calculate average salary, average male salary, average female salary, and wage gap
windowSpec = Window.partitionBy("JobTitle")

result = df.withColumn("Avg_Salary", avg(col("Salary($)")).over(windowSpec)) \
    .withColumn("Avg_Male_Salary", avg(when(col("Gender") == "Male", col("Salary($)"))).over(windowSpec)) \
    .withColumn("Avg_Female_Salary", avg(when(col("Gender") == "Female", col("Salary($)"))).over(windowSpec)) \
    .withColumn("Wage_Gap", col("Avg_Male_Salary") - col("Avg_Female_Salary"))

# Show the result
result.show()


In [ ]:
# Query 3 : Gender wage gaps by education level

# Calculate average salary, average male salary, average female salary, and wage gap
windowSpec = Window.partitionBy("Education")

result = df.withColumn("Avg_Salary", avg(col("Salary($)")).over(windowSpec)) \
    .withColumn("Avg_Male_Salary", avg(when(col("Gender") == "Male", col("Salary($)"))).over(windowSpec)) \
    .withColumn("Avg_Female_Salary", avg(when(col("Gender") == "Female", col("Salary($)"))).over(windowSpec)) \
    .withColumn("Wage_Gap", col("Avg_Male_Salary") - col("Avg_Female_Salary"))

# Show the result
result.show()


In [ ]:
# Query 4 : Salary increase by years of experience

# Calculate average salary and salary increase
windowSpec = Window.partitionBy("Dept", "Seniority").orderBy("Seniority")

result = df.withColumn("Avg_Salary", avg(col("Salary($)")).over(windowSpec)) \
    .withColumn("Salary_Increase", avg(col("Salary($)")).over(windowSpec) - lag(col("Avg_Salary")).over(windowSpec)) \
    .groupBy("Seniority", "Dept", "Salary_Increase") \
    .agg(avg(col("Salary($)")).alias("Avg_Salary"))

# Show the result
result.show()



In [ ]:
# Query 5 : Salary growth rate by years of work experience

windowSpec = Window.partitionBy("Dept").orderBy("Seniority")

result = df.withColumn("Avg_Salary", avg(col("Salary($)").cast("double")).over(windowSpec)) \
    .withColumn("Salary_Growth_Rate", ((col("Avg_Salary") - lag(col("Avg_Salary")).over(windowSpec)) / lag(col("Avg_Salary")).over(windowSpec)).cast("double") * 100) \
    .select("Seniority", "Dept", "Avg_Salary", "Salary_Growth_Rate") \
    .groupBy("Seniority", "Dept", "Avg_Salary", "Salary_Growth_Rate") \
    .agg(F.max("Salary_Growth_Rate").alias("Max_Salary_Growth_Rate"))

result.show()




In [ ]:
# Query 6 : Salary distribution by years of work experience

# Register the DataFrame as a temporary table
df.createOrReplaceTempView("GlassdoorGenderTable")

# Perform the query
query = """
SELECT Seniority,
       Dept,
       `Salary($)` AS Salary,
       NTILE(4) OVER (PARTITION BY Dept ORDER BY Seniority) AS Quartile
FROM GlassdoorGenderTable
"""
result = spark.sql(query)

# Show the result
result.show()

